<a href="https://colab.research.google.com/github/hindia-vic/transformers/blob/main/named_entity_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U transformers
!pip install -U datasets
!pip install -U accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 117.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.4
    Uninstalling transformers-4.52.4:
      Successfully uninstalled transformers-4.52.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Lin

In [ ]:
import pandas as pd
from datasets import load_dataset

In [ ]:
data=load_dataset('conll2003')
data

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [ ]:
data['train'].features

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'pos_tags': Sequence(feature=ClassLabel(names=['"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'NN|SYM', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB'], id=None), length=-1, id=None),
 'chunk_tags': Sequence(feature=ClassLabel(names=['O', 'B-ADJP', 'I-ADJP', 'B-ADVP', 'I-ADVP', 'B-CONJP', 'I-CONJP', 'B-INTJ', 'I-INTJ', 'B-LST', 'I-LST', 'B-NP', 'I-NP', 'B-PP', 'I-PP', 'B-PRT', 'I-PRT', 'B-SBAR', 'I-SBAR', 'B-UCP', 'I-UCP', 'B-VP', 'I-VP'], id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)}

In [ ]:
pd.DataFrame(data['train'][:5])[['tokens','ner_tags']]

,tokens,ner_tags
0,"[EU, rejects, German, call, to, boycott, Briti...","[3, 0, 7, 0, 0, 0, 7, 0, 0]"
1,"[Peter, Blackburn]","[1, 2]"
2,"[BRUSSELS, 1996-08-22]","[5, 0]"
3,"[The, European, Commission, said, on, Thursday...","[0, 3, 4, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, ..."
4,"[Germany, 's, representative, to, the, Europea...","[5, 0, 0, 0, 0, 3, 4, 0, 0, 0, 1, 2, 0, 0, 0, ..."


In [ ]:
tags=data['train'].features['ner_tags']
indextotags={i:tag for i,tag in enumerate(tags.feature.names)}
tagstoindex={tag:i for i,tag in enumerate(tags.feature.names)}

In [ ]:
def tag_names(batch):
  tag_name={'ner_tags_str':[tags.feature.int2str(idx) for idx in batch['ner_tags']]}
  return tag_name

In [ ]:
data=data.map(tag_names)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'ner_tags_str'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'ner_tags_str'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'ner_tags_str'],
        num_rows: 3453
    })
})

In [ ]:
pd.DataFrame(data['train'][:5])[['tokens','ner_tags','ner_tags_str']]

,tokens,ner_tags,ner_tags_str
0,"[EU, rejects, German, call, to, boycott, Briti...","[3, 0, 7, 0, 0, 0, 7, 0, 0]","[B-ORG, O, B-MISC, O, O, O, B-MISC, O, O]"
1,"[Peter, Blackburn]","[1, 2]","[B-PER, I-PER]"
2,"[BRUSSELS, 1996-08-22]","[5, 0]","[B-LOC, O]"
3,"[The, European, Commission, said, on, Thursday...","[0, 3, 4, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, ...","[O, B-ORG, I-ORG, O, O, O, O, O, O, B-MISC, O,..."
4,"[Germany, 's, representative, to, the, Europea...","[5, 0, 0, 0, 0, 3, 4, 0, 0, 0, 1, 2, 0, 0, 0, ...","[B-LOC, O, O, O, O, B-ORG, I-ORG, O, O, O, B-P..."


In [ ]:
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [ ]:
tokenizer.is_fast

True

In [ ]:
def align_labels(labels,word_ids):
  new_labels=[]
  current_word=None
  for word_id in word_ids:
    if word_id!=current_word:
      current_word=word_id
      label=-100 if word_id is None else labels[word_id]
      new_labels.append(label)
    elif word_id is None:
      new_labels.append(-100)
    else:
      label=labels[word_id]
      if label%2==1:
        label+=1
      new_labels.append(label)
  return new_labels

In [ ]:
labels=data['train'][0]['ner_tags']
labels

[3, 0, 7, 0, 0, 0, 7, 0, 0]

In [ ]:
inputs=tokenizer(data['train'][0]['tokens'],is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'eu',
 'rejects',
 'german',
 'call',
 'to',
 'boycott',
 'british',
 'lamb',
 '.',
 '[SEP]']

In [ ]:
words_id=inputs.word_ids()
print(labels,words_id)

[3, 0, 7, 0, 0, 0, 7, 0, 0] [None, 0, 1, 2, 3, 4, 5, 6, 7, 8, None]


In [ ]:
align_labels(labels,words_id)

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100]

In [ ]:
def tokenize_and_align_labels(examples):
  tokenized_inputs=tokenizer(examples['tokens'],truncation=True,is_split_into_words=True)
  all_labels=examples['ner_tags']
  new_labels=[]
  for i,labels in enumerate(all_labels):
    word_ids=tokenized_inputs.word_ids(i)
    new_labels.append(align_labels(labels,word_ids))
  tokenized_inputs['labels']=new_labels
  return tokenized_inputs

In [ ]:
tokenized_data=data.map(tokenize_and_align_labels,batched=True,remove_columns=data['train'].column_names)
tokenized_data

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})

In [ ]:
from transformers import DataCollatorForTokenClassification
data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
tokenized_data['train'][0]

{'input_ids': [101,
  7327,
  19164,
  2446,
  2655,
  2000,
  17757,
  2329,
  12559,
  1012,
  102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100]}

In [ ]:
batch=data_collator([tokenized_data['train'][i] for i in range(2)])
batch['labels']

tensor([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0, -100],
        [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100]])

In [ ]:
from transformers import DataCollatorForTokenClassification
data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer)
batch=data_collator([tokenized_data['train'][i] for i in range(2)])
batch['labels']

tensor([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0, -100],
        [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100]])

In [ ]:
!pip install seqeval
!pip install evaluate

In [ ]:
import evaluate
metric=evaluate.load('seqeval')

In [ ]:
ner_feature=data['train'].features['ner_tags']
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [ ]:
labels_name=ner_feature.feature.names
labels_name

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [ ]:
labels=data['train'][0]['ner_tags']
labels=[labels_name[i] for i in labels]
labels

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [ ]:
prediction=labels.copy()
prediction[2]='O'
metric.compute(predictions=[prediction],references=[labels])

{'MISC': {'precision': np.float64(1.0),
  'recall': np.float64(0.5),
  'f1': np.float64(0.6666666666666666),
  'number': np.int64(2)},
 'ORG': {'precision': np.float64(1.0),
  'recall': np.float64(1.0),
  'f1': np.float64(1.0),
  'number': np.int64(1)},
 'overall_precision': np.float64(1.0),
 'overall_recall': np.float64(0.6666666666666666),
 'overall_f1': np.float64(0.8),
 'overall_accuracy': 0.8888888888888888}

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
  logits,labels=eval_pred
  predictions=np.argmax(logits,axis=-1)
  true_labels=[[labels_name[l]for l in label if l!=-100 and 0 <= l < len(labels_name)]for label in labels]
  true_predictions=[[labels_name[p]for p,l in zip(prediction,label) if l!=-100 and 0 <= p < len(labels_name)]for prediction,label in zip(predictions,true_labels)]
  results=metric.compute(predictions=true_predictions,references=true_labels)
  return {
      'precision':results['overall_precision'],
      'recall':results['overall_recall'],
      'f1':results['overall_f1'],
      'accuracy':results['overall_accuracy']
  }

In [ ]:
id2label={i:label for i,label in enumerate(labels_name)}
label2id={label:i for i,label in enumerate(labels_name)}

In [ ]:
from transformers import AutoModelForTokenClassification
model=AutoModelForTokenClassification.from_pretrained(
    'distilbert-base-uncased',num_labels=len(tags.feature.names),id2label=id2label,label2id=label2id
)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.config.num_labels

9

In [ ]:
from transformers import TrainingArguments
training_args=TrainingArguments(
    output_dir='distillbert_ner',
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to='none'
)

In [ ]:
from transformers import Trainer
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data['train'],
    eval_dataset=tokenized_data['test'],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

/tmp/ipython-input-71-2844647544.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer=Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.076400,0.111474,0.001043,0.001062,0.001053,0.745995
2,0.034900,0.125767,0.000703,0.000708,0.000706,0.747721
3,0.022900,0.128542,0.000874,0.000885,0.000880,0.747756


TrainOutput(global_step=5268, training_loss=0.06595009370744455, metrics={'train_runtime': 393.9129, 'train_samples_per_second': 106.935, 'train_steps_per_second': 13.374, 'total_flos': 445994355589020.0, 'train_loss': 0.06595009370744455, 'epoch': 3.0})

In [ ]:
from transformers import pipeline
model_checkpoint='/content/distillbert_ner/checkpoint-5268'
token_classifier=pipeline(
    'token-classification',model=model_checkpoint,aggregation_strategy='simple',device=0
)
token_classifier("my name is victor hindia i live in kenya. I work at Snettscom Innovative Solution")


Device set to use cuda:0


[{'entity_group': 'PER',
  'score': np.float32(0.99794406),
  'word': 'victor hindia',
  'start': 11,
  'end': 24},
 {'entity_group': 'LOC',
  'score': np.float32(0.99861026),
  'word': 'kenya',
  'start': 35,
  'end': 40},
 {'entity_group': 'ORG',
  'score': np.float32(0.98522305),
  'word': 'snettscom innovative solution',
  'start': 52,
  'end': 81}]